# Tutorial 6.2. Extreme value analysis - 2

### Description: The wind effects on building structure according to EN 1991-1-4 is presented in this exercise. Along wind and across wind response of building structure according to EN 1991-1-4 and German national annexe is also presented. Some exercises are proposed. 

#### Students are advised to complete the exercises. 

Project: Structural Wind Engineering WS19-20 
        Chair of Structural Analysis @ TUM - R. Wüchner, M. Péntek
        
Author: anoop.kodakkal@tum.de, mate.pentek@tum.de

Created on:  30.11.2015

Last update: 06.12.2019

##### Contents:

 1. Along wind response of structure
 2. Vortex shedding checks 
 3. Instability - Galloping 

### Codes and standards :
#### 1. EN 1991-1-4:2005 - Eurocode 1: Actions on structures - Part 1-4: General actions - Wind actions
#### 2.  DIN EN 1991-1-4/NA:2010-12 : Nationaler Anhang  EN 1991-1-4

Both can be downloaded from the links available in Moodle. It is advised to have them with you through out this exercise. 

In [1]:
# import
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive

## 1. Along wind response - computation of $c_s c_d$


#### Gust wind speed computed from the previous example for the location of the Jeddah Airport considering a return period of 50 years is 40.12 m/s . The mean wind speed is computed as $$ u_{mean} = u_{gust}/1.4$$  
The building is located at an urban area with height of adjacent building upto 15m: __Terrain category IV__

Let us take the same building as in previous example  

height : 600 m 

uniform cross section of given geometry and building width = 60.0 m

Assume a structural damping of 5% 
Reference height be 200 m

In [2]:
mean_windspeed = 40.12 / 1.4
height_ref = 200
height = 600
width = 50
damping = 0.005
frequency_1 = 46 / height # refer to F.2

According to DIN EN 1991-1-4/NA the wind profile for terrain category IV is   __ref: Tabelle NA.B.2__

$$ u_{mean}(z) = 0,56 . v_b . (z/10)^{0.3}$$ 

$$ u_{gust}(z) = 1,05 . v_b . (z/10)^{0.2}$$ 

Turbulence intensity $$ I_{v}(z) = 0,43 . (z/10)^{-0.3}$$ 

In [3]:
terrain_category = "II"

a_mean  = {'I':1.18,'II':1.0,'III':0.77,'IV':0.56 }
alpha_mean = {'I':0.12,'II':0.16,'III':0.22,'IV':0.3 }
a_gust = {'I':1.61,'II':1.45,'III':1.27,'IV':1.05 }
alpha_gust = {'I':0.095,'II':0.12,'III':0.155,'IV':0.2 }
a_turb = {'I':0.14,'II':0.19,'III':0.28,'IV':0.43 }
alpha_turb = {'I':-0.12,'II':-0.16,'III':-0.22,'IV':-0.3 }

In [4]:
umean = a_mean[terrain_category] * mean_windspeed * (height_ref/10)**alpha_mean[terrain_category]
ugust = a_gust[terrain_category] * mean_windspeed * (height_ref/10)**alpha_gust[terrain_category]
Iv = a_turb[terrain_category]  * (height_ref/10)**alpha_turb[terrain_category]
print('Turbulence Intensity', np.around(Iv,3))

Turbulence Intensity 0.118


###### Computation of length scale L(z) __ref:  NA.C.1.2 & ref: Tabelle NA.C.1__ . 

Length scale for terrain category IV  $$ L(z) = 300 .(z/300)^{0,46}$$ 

In [5]:

alpha_lengthscale = {'I':0.13,'II':0.26,'III':0.37,'IV':0.46}
len_scale = 300 * (height_ref/300) ** alpha_lengthscale[terrain_category]
print(len_scale)

269.9836891311147


###### Computation of Non-dimensional power spectral density function SL(z,n) __ref:  NA.C.1.2 __ . 

Non-dimensional power spectral density function S_L(z,n) $$ S_L(z, n) = \dfrac{6,8 . f_L (z,n)}{(1+10,2 . f_L(z,n))^{5/3}}$$

where $f_L(z,n) = \dfrac{n  L(z)}{u_{mean}(z)}$ is the non-dimensional frequency

In [6]:
f_l = frequency_1 * len_scale / umean
S_l = 6.8 * f_l / (1+10.2 * f_l)** (5/3)
print(S_l)

0.17418693195975118


#### Determination of the Background factor B² __ref: B.2 __ . 

the background factor can be calculated as  $$ B^2 = \dfrac{1}{1+0,9 . \bigg( \frac {b+h}{L(z)}\bigg)^{0,63}}$$


In [7]:
bg_factor = 1/(1+0.9*((width + height)/len_scale)**0.63)
print(bg_factor)

0.3897973090370115


###### Determination of the Resonance factor R² __ref: B.2 __

the Resonance factor can be calculated as  $$ R^2 = \dfrac{\pi^2}{2.\delta} . S_L(z,n). R_h(\eta_h). R_b(\eta_b) $$

please refer to equation B.7 B.8 and B.5 


In [8]:
eta_h = 4.6 * height / len_scale * f_l
eta_b = 4.6 * width / len_scale * f_l 
R_h = 1/eta_h - 1/2/eta_h**2 * ( 1 - np.e ** (-2 * eta_h))
R_b = 1/eta_b - 1/2/eta_b**2 * ( 1 - np.e ** (-2 * eta_b))
log_decrement = damping * 2 * np.pi
res_factor = np.pi**2 /2/log_decrement * S_l * R_h * R_b
up_cross_freq = max(frequency_1 * np.sqrt( res_factor / (res_factor + bg_factor)),0.08)
print(res_factor)

4.199238753019387


###### Determination of the peak factor kp __ref: B.2 __

please refer to equation B.4

In [9]:
peak_factor = max(3, np.sqrt(2 * np.log(up_cross_freq * 600 )) + 0.6 / (np.sqrt(2*np.log(up_cross_freq * 600))))
print(peak_factor)

3


###### Determination of structural factor $C_sC_d$

please refer to equation NA.C.1 
 $$ c_s c_d = \dfrac{1+2 . k_p . I_v(z) . \sqrt{B^2 + R^2 } } {1+6. I_v(z)} $$

In [10]:
structural_factor = (1 + 2*peak_factor*Iv*np.sqrt(bg_factor+res_factor))/(1+6 * Iv)
print('Structural factor c_s c_d is ', np.around(structural_factor, 3))

Structural factor c_s c_d is  1.473


### Exercise: Where is the $c_sc_d$ is used ? 

Refer to equation 5.4 

For the previous exercise 3_1 , recompute the bending moment and shear force considering this factor.
Knowing the structural properties compute the static displacements for this case. 

## 2. Vortex shedding 

Please refer to Annex E 

The effects of Vortex shedding need to be investigated when $V_{crit} < 1,25 . V_m$


The critical wind velocity is $V_{crit} = \dfrac{b . n_{i,y}}{St}$

Refer to E1.3.1 for details 

In [11]:
strouhal_number = 0.13 # for rectangular cross section # refer to E1.3.2 for details 

crosswind_freq = 0.26 # this refers to the frequency correspondign to th bending mode in crosswind direction
velocity_crit = width * crosswind_freq / strouhal_number

In [12]:
vortex_freq = strouhal_number * umean / width
print('Vortex shedding Frequency ', np.around(vortex_freq,3))

Vortex shedding Frequency  0.12


In [13]:
if (1.25 * umean > velocity_crit): 
    print('NOt OK, The effect of vortex shedding need to be investigated')
else: 
        print('OK, The effect of vortex shedding need to be investigated')

OK, The effect of vortex shedding need to be investigated


##### Calculation of cross wind amplitude of displacements

ref E.1.5.2.1 : the largest displacement $y_{F,max}$ can be calculated by the expression (E.7) as 

$y_{F,max} = \dfrac{1}{St^2}.\dfrac{1}{S_C}.K.K_w.c_{lat}$

where, $S_C$ is the Scruton number as in E.1.3.3. 
Please refer to section E.1.5.2 for more details

## 3. Galloping

Please refer to Annex E.2

The onset wind velocity of galloping is given by $$ V_{cg} = \dfrac{2.Sc}{a_G} . n_{1y}.b$$

#### Discuss among groups the different wind effects on structure 